In [3]:
import pandas as pd
import joblib
import numpy as np
import os

# --- KONFIGURASI: Ganti dengan path file Anda ---
MODEL_PATH = 'waste_model.joblib' 
SCALER_PATH = 'scaler.joblib'
TRAINING_DATA_PATH = 'data_provinsi.csv' 
OUTPUT_PATH = os.path.join('public', 'predictions_2025.csv')
# ------------------------------------------------

# Fungsi untuk memeriksa keberadaan file
def check_file_exists(path):
    if not os.path.exists(path):
        print(f"[ERROR] File tidak ditemukan di path: '{os.path.abspath(path)}'.")
        print("Pastikan Anda menjalankan skrip ini dari direktori root proyek 'WasteSnap' dan path file sudah benar.")
        return False
    return True

# Validasi semua path input
if not all([check_file_exists(p) for p in [MODEL_PATH, SCALER_PATH, TRAINING_DATA_PATH]]):
    exit()

# Muat model, scaler, dan data training
rf_model = joblib.load(MODEL_PATH)
scaler = joblib.load(SCALER_PATH)
df_train = pd.read_csv(TRAINING_DATA_PATH)

# --- INI ADALAH BAGIAN KUNCI UNTUK MEREPLIKASI PROSES TRAINING ---

# 1. Pisahkan fitur (X) dari data training untuk membuat template
X_train_template = df_train.drop(columns=['persentase'])

# 2. Definisikan kolom kategori secara eksplisit
categorical_features = ['nama_provinsi', 'jenis_sampah']

# 3. Terapkan one-hot encoding dengan drop_first=True, SAMA SEPERTI SAAT TRAINING
#    Ini adalah "golden standard" kolom kita.
X_train_encoded_template = pd.get_dummies(X_train_template, columns=categorical_features, drop_first=True) # <-- PERUBAHAN KUNCI
X_columns_standard = X_train_encoded_template.columns

# 4. Dapatkan daftar unik provinsi dan jenis sampah untuk iterasi
unique_provinces = sorted(list(df_train['nama_provinsi'].unique()))
unique_waste_types = sorted(list(df_train['jenis_sampah'].unique()))

print("Struktur kolom yang diharapkan oleh model telah berhasil dibuat.")
print(f"Total kolom: {len(X_columns_standard)}")
print("\nMemulai proses prediksi untuk semua provinsi...")

all_predictions = []
prediction_year = 2025

# Lakukan prediksi untuk setiap kombinasi
for province_name in unique_provinces:
    for waste_type in unique_waste_types:
        # Buat DataFrame input mentah untuk satu baris prediksi
        raw_input_df = pd.DataFrame([{'tahun': prediction_year, 'nama_provinsi': province_name, 'jenis_sampah': waste_type}])
        
        # Terapkan one-hot encoding DENGAN CARA YANG SAMA PERSIS (drop_first=True)
        processed_input = pd.get_dummies(raw_input_df, columns=categorical_features, drop_first=True) # <-- PERUBAHAN KUNCI
        
        # REINDEX: Paksa DataFrame prediksi agar memiliki kolom yang persis sama
        final_input = processed_input.reindex(columns=X_columns_standard, fill_value=0)
        
        # Prediksi dengan model
        rf_pred_scaled = rf_model.predict(final_input)[0]
        
        # Kembalikan nilai prediksi ke skala aslinya
        rf_pred_original = scaler.inverse_transform(np.array([[rf_pred_scaled]]))[0][0]
        
        # Tambahkan hasil prediksi ke daftar
        all_predictions.append({
            'tahun': prediction_year,
            'nama_provinsi': province_name,
            'jenis_sampah': waste_type,
            'persentase': round(rf_pred_original, 4)
        })

# Buat DataFrame dari semua prediksi
df_predictions = pd.DataFrame(all_predictions)

# Pastikan direktori 'public' ada
os.makedirs('public', exist_ok=True)

# Simpan ke file CSV di folder public
df_predictions.to_csv(OUTPUT_PATH, index=False)

print(f"\n[SUCCESS] Prediksi berhasil dibuat dan disimpan di '{OUTPUT_PATH}'")
print("File ini siap digunakan oleh aplikasi React Anda.")

Struktur kolom yang diharapkan oleh model telah berhasil dibuat.
Total kolom: 42

Memulai proses prediksi untuk semua provinsi...

[SUCCESS] Prediksi berhasil dibuat dan disimpan di 'public\predictions_2025.csv'
File ini siap digunakan oleh aplikasi React Anda.
